In [1]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau, TensorBoard
from keras.models import Model
import keras.backend as K

import os
import zipfile
import numpy as np

from google.colab import drive
drive.mount('/content/drive')



Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
datapath = '/content/drive/My Drive/Code_Attribution'
zipPaths = []
for r, d, f in os.walk(datapath):
  for name in f:
    zipPaths.append(os.path.join(r, name))
    
    
for p in zipPaths:
  zip_ref = zipfile.ZipFile(p, 'r')
  zip_ref.extractall('/content/alldata')
  zip_ref.close()

In [0]:
datasetPath = '/content/alldata'

In [0]:
path = datasetPath

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.cpp' in file:
            files.append(os.path.join(r, file))

# for f in files:
#     print(f)

In [5]:
print(len(files))
count = 0
for p in files:
  if not (os.path.isdir(p)):
    count += 1

print(count)

384125
384125


In [0]:
minLength = 2000

In [0]:
def padding(fname):
    f = open(fname, "r")
    y = f.read()
    f.close()
    
    origy = y
    while (len(y) < minLength):
        y += origy
        
    f = open(fname, "w")
    f.write(y)
    f.close()
    return y

In [0]:
outputvar = {}
counter = [0]

In [0]:
x_train = []
x_test = []
y_train = []
y_test = []
x_val = []
y_val = []
split = 0.8

In [16]:
for i in range(0,4):
    print(i)
    lengths = []
    count = 0
    for fil in files:
        f = open(fil, "r")
        x = f.read()
        y = x.split()
        f.close()
        if(len(x) <= 100):
            files.remove(fil)
        elif len(x) == 0:
            files.remove(fil)
        elif(len(y) < minLength):
            x = padding(fil)
            lengths.append(len(x))
        else:
            lengths.append(len(x))


0
1
2
3


In [17]:
count = 0

for fil in files:
  f = open(fil, "r")
  x = f.read()
  y = x.split()
  f.close()
  if(len(x) < 2000):
      print(fil)
      count += 1
  elif len(x) == 0:
      print(fil)
      count += 1
      
print(count, ' files remaining less than 2000.')


0  files remaining less than 2000.


In [18]:
print(max(lengths))
print(min(lengths))
print(len(lengths))
print(len(files))

102387
2000
383669
383669


In [0]:
def makeInOut(path, _len, counter):
  count = 0
  for r, d, f in os.walk(path):
    splitted = r.split('/')
    name = splitted[-1]+splitted[-2]
    for fil in f:
      _fil = os.path.join(r, fil)
      if name not in outputvar.keys():
        outputvar[name] = counter[0]
        counter[0] += 1
      if _fil in files:
        if count < (0.8*_len):
          if count < (0.2*_len):
            x_val.append(_fil)
            y_val.append(outputvar[name])
          else:
            x_train.append(_fil)
            y_train.append(outputvar[name])
          count += 1
        else:
          x_test.append(_fil)
          y_test.append(outputvar[name])

In [0]:
def rem_dir(path):
  for r, d, f in os.walk(path):
    for fil in f:
      if fil in files:
        files.remove(os.path.join(r, fil))

In [0]:
def check(path, counter):
  for r, d, f in os.walk(path):
    _len = len(f)
    if _len < 5:
      rem_dir(path)
    else:
      makeInOut(path, _len, counter)

In [22]:
def checkDir(dirPath):
  for r, d, f in os.walk(dirPath):
    for di in d:
      check(os.path.join(r, di), counter)
      
count = 0

directories = os.listdir(datasetPath)
for x in directories:
  checkDir(str(datasetPath+'/'+x))
  print(x + ' done')

#   if count >= 5:
#     break
#   count += 1
#   for d in d1:
#     print(os.path.join(r1, d))
#     count += 1


data11 done
data16 done
data09 done
data13 done
data17 done
data10 done
data14 done
data15 done
data08 done
data12 done


In [26]:
_check = []
for x in y_train:
  if x not in _check:
    _check.append(x)
    
print(len(_check))
print(counter)
print(len(outputvar))

35245
[35263]
35263


In [28]:
for x in y_test:
  if x not in _check:
    print(x)
    
for x in y_val:
  if x not in _check:
    print(x)

1521
1521
27009
34888


In [24]:
len(files)

383669

In [25]:
print(max(lengths))
print(min(lengths))
print(len(lengths))
print(len(files))

102387
2000
383669
383669


In [29]:
print(x_train[:20])
print(x_test[:10])
print(y_train[:30])
print(y_test[:10])
print(x_val[:10])
print(y_val[:10])

['/content/alldata/data11/M.H./b0.cpp', '/content/alldata/data11/M.H./a0.cpp', '/content/alldata/data11/M.H./b1.cpp', '/content/alldata/data11/TarifEzaz/codejamA0.cpp', '/content/alldata/data11/TarifEzaz/code1c0.cpp', '/content/alldata/data11/TarifEzaz/jammA0.cpp', '/content/alldata/data11/TarifEzaz/code3c0.cpp', '/content/alldata/data11/TarifEzaz/code1c1.cpp', '/content/alldata/data11/TarifEzaz/jammA1.cpp', '/content/alldata/data11/WtbH/A1.cpp', '/content/alldata/data11/WtbH/D0.cpp', '/content/alldata/data11/WtbH/C0.cpp', '/content/alldata/data11/WtbH/B1.cpp', '/content/alldata/data11/WtbH/C1.cpp', '/content/alldata/data11/Raulpitz/gcj20.cpp', '/content/alldata/data11/Raulpitz/gcj21.cpp', '/content/alldata/data11/Raulpitz/gcj330.cpp', '/content/alldata/data11/nihao/p30.cpp', '/content/alldata/data11/nihao/p10.cpp', '/content/alldata/data11/nihao/p20.cpp']
['/content/alldata/data11/M.H./a1.cpp', '/content/alldata/data11/TarifEzaz/codejamA1.cpp', '/content/alldata/data11/TarifEzaz/codej

In [0]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_val = np.array(x_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [31]:
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

indices = np.arange(x_test.shape[0])
np.random.shuffle(indices)
x_test = x_test[indices]
y_test = y_test[indices]

indices = np.arange(x_val.shape[0])
np.random.shuffle(indices)
x_val = x_val[indices]
y_val = y_val[indices]

print(x_train[:20])
print(x_test[:10])
print(y_train[:30])
print(y_test[:10])
print(x_val[:10])
print(y_val[:10])

['/content/alldata/data09/Shark/a1.cpp'
 '/content/alldata/data14/Paja/a0.cpp'
 '/content/alldata/data15/MarioYC/C0.cpp'
 '/content/alldata/data08/Cheryl/p20.cpp'
 '/content/alldata/data12/square116/a0.cpp'
 '/content/alldata/data17/alexMcCool/A1.cpp'
 '/content/alldata/data14/jargon/p20.cpp'
 '/content/alldata/data17/Denisson/winnow0.cpp'
 '/content/alldata/data08/kobi2000/a0.cpp'
 '/content/alldata/data17/crbonilha/pony0.cpp'
 '/content/alldata/data17/myblueluck/C0.cpp'
 '/content/alldata/data17/freaky.ju/Codejam_qualificationround2017_Qb1.cpp'
 '/content/alldata/data13/hesibo/a0.cpp'
 '/content/alldata/data16/NaiveRed/D0.cpp'
 '/content/alldata/data17/johnchen902/C0.cpp'
 '/content/alldata/data08/lzz/C0.cpp'
 '/content/alldata/data10/zxytim/Elegant Diamond1.cpp'
 '/content/alldata/data10/daenohuang/GCJ21.cpp'
 '/content/alldata/data14/Venox/goo0511a1.cpp'
 '/content/alldata/data13/enzneer/tictoe1.cpp']
['/content/alldata/data16/Superk55/Prob20.cpp'
 '/content/alldata/data14/Carber/Q

In [0]:
def walk(path, vocab, count):
    for r, d, f in os.walk(path):
        for file in f:
            f = open(os.path.join(r, file))
            x = f.read()
            for line in x:
                for letter in line:
                    if letter not in vocab.keys():
                        vocab[letter] = count[0]
                        count[0] += 1


In [33]:
vocab = {}
count = [0]
path = datasetPath
for r, d, f, in (os.walk(path)):
    for i, dir in enumerate(d):
        walk(os.path.join(r, dir), vocab, count)
print(count)

[3549]


In [34]:
i = 0
for word in vocab.keys():
  print(word)
  i += 1
  if i >= 10:
    break

#
i
n
c
l
u
d
e
 
<


In [0]:
def getFiles(path):
    matrix = []
    for p in path:
        tmp = np.zeros((50, 40), dtype = int)
        f = open(p)
        for i in range(0,50):
            for j in range(0, 40):
                tmp[i][j] = vocab[f.read(1)]
        matrix.append(tmp)
    return np.array(matrix)

In [0]:
num_classes = len(outputvar)
input_shape = (50, 40)
batch_size = 1024
def data_generator(file_paths, author_numbers):
    batch_start = 0
    batch_end = batch_start + batch_size
    n = file_paths.shape[0]
    indexes = np.arange(0, n, batch_size)
    if n % batch_size != 0:
        indexes = indexes[:-1] 
    while True:
        np.random.shuffle(indexes)
        for ind in indexes:
            batch_start = ind
            batch_end = batch_start + batch_size
            myFiles = file_paths[batch_start:batch_end]
            batch_files = getFiles(myFiles)
            batch_authors = author_numbers[batch_start:batch_end]
            batch_x = np.zeros((batch_size, 50, 40))
            tmp = np.zeros((batch_size, num_classes))
            batch_x = np.array( batch_files )
            for i, j in enumerate(batch_authors):
                tmp[i][j] = 1
            batch_y = np.array( tmp )
            yield( batch_x, batch_y )

In [37]:
input_fil = Input(shape=(input_shape), name='input_fil')
conv = Conv1D(64, kernel_size=7, strides = 2,  activation='relu')(input_fil)
drop = Dropout(0.1)(conv)
maxP = MaxPooling1D()(drop)
flat = Flatten()(maxP)
output_class = Dense(num_classes, activation='softmax')(flat)

model = Model(inputs=input_fil, outputs=output_class)

W0728 20:14:35.503359 140116532483968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 20:14:35.554412 140116532483968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 20:14:35.556585 140116532483968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 20:14:35.593576 140116532483968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0728 20:14:35.601210 

In [38]:
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])

model.summary()

W0728 20:14:38.478607 140116532483968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 20:14:38.487496 140116532483968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_fil (InputLayer)       (None, 50, 40)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 64)            17984     
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 704)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 35263)             24860415  
Total params: 24,878,399
Trainable params: 24,878,399
Non-trainable params: 0
________________________________________________________________

In [0]:
train_gen = data_generator(x_train, y_train)
test_gen = data_generator(x_test, y_test)
val_gen = data_generator(x_val, y_val)

In [40]:
hist1 = model.fit_generator(train_gen, epochs=10, steps_per_epoch=len(y_train)//batch_size, 
                           validation_data=val_gen, validation_steps=len(y_val)//batch_size, verbose=1)

W0728 20:14:43.156357 140116532483968 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
157/157 [==============================] - 290s 2s/step - loss: 10.7100 - acc: 0.0039 - val_loss: 10.0324 - val_acc: 0.0198
Epoch 2/10
157/157 [==============================] - 268s 2s/step - loss: 8.2549 - acc: 0.0697 - val_loss: 9.9351 - val_acc: 0.0695
Epoch 3/10
157/157 [==============================] - 269s 2s/step - loss: 5.9102 - acc: 0.2197 - val_loss: 9.9322 - val_acc: 0.1373
Epoch 4/10
157/157 [==============================] - 269s 2s/step - loss: 4.3607 - acc: 0.3666 - val_loss: 9.9634 - val_acc: 0.1772
Epoch 5/10
157/157 [==============================] - 268s 2s/step - loss: 3.2230 - acc: 0.4970 - val_loss: 9.8727 - val_acc: 0.2392
Epoch 6/10
157/157 [==============================] - 267s 2s/step - loss: 2.2776 - acc: 0.6215 - val_loss: 9.6643 - val_acc: 0.2856
Epoch 7/10
157/157 [==============================] - 268s 2s/step - loss: 1.4636 - acc: 0.7490 - val_loss: 9.3836 - val_acc: 0.3394
Epoch 8/10
157/157 [==============================] - 268s 2s/step 

In [41]:
xtestdata = getFiles(x_test)
print(xtestdata.shape)

(43767, 50, 40)


In [42]:
preds = model.predict(xtestdata)
print(preds.shape)

(43767, 35263)


In [43]:
myPreds = []
for i in preds:
    myPreds.append(np.argmax(i))
myPreds = np.array(myPreds)
print(myPreds.shape)

(43767,)


In [44]:
count = 0
for i in range(myPreds.shape[0]):
    if myPreds[i] == y_test[i]:
        count += 1

acc = (count / myPreds.shape[0]) * 100
print("Accuracy", acc)

Accuracy 41.81460918043275
